In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import joblib

data = pd.read_csv('C:/Users/RAJ/Desktop/PY/real-estate-predictor/data/new_housing.csv')

x = data[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']]
y = data['median_house_value']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(x_train, y_train)

prediction = model.predict(x_test)

mse = mean_squared_error(y_test, prediction)
print(f"mean squared error : {mse}")

joblib.dump(model, 'house_model.pkl')

print("Model saved successfully")

mean squared error : 4025735197.027956
Model saved successfully
